In [25]:
from dotenv import load_dotenv
import os
import pinecone

# Load environment variables from a .env file
load_dotenv(dotenv_path='.env.local')





True

In [26]:
pc = pinecone.Pinecone(os.getenv("PINECONE_API_KEY"))
try:
    pc.create_index(
        name="rag",
        dimension = 1024,
        metric="cosine",
        spec=pinecone.ServerlessSpec(cloud="aws", region="us-east-1")
    )
except:
    pass 

In [27]:
import json
data = json.load(open("reviews.json"))

data['reviews']

[{'title': 'Inception',
  'genre': 'Sci-Fi, Thriller',
  'stars': 5,
  'reviewer': 'Alice Johnson',
  'comments': 'Mind-bending and visually stunning. A masterpiece by Christopher Nolan.'},
 {'title': 'The Dark Knight',
  'genre': 'Action, Crime, Drama',
  'stars': 5,
  'reviewer': 'Bob Smith',
  'comments': "A brilliant portrayal of Batman. Heath Ledger's Joker is unforgettable."},
 {'title': 'Interstellar',
  'genre': 'Adventure, Drama, Sci-Fi',
  'stars': 4,
  'reviewer': 'Carol Williams',
  'comments': 'A visually impressive journey through space, but the plot can be a bit complex.'},
 {'title': 'The Shawshank Redemption',
  'genre': 'Drama',
  'stars': 5,
  'reviewer': 'David Brown',
  'comments': 'A timeless story of hope and friendship. One of the best films ever made.'},
 {'title': 'The Godfather',
  'genre': 'Crime, Drama',
  'stars': 5,
  'reviewer': 'Eva Green',
  'comments': 'A cinematic masterpiece with powerful performances and a gripping story.'},
 {'title': 'Pulp Fictio

In [28]:
from mixedbread_ai.client import MixedbreadAI
processed_data=[]

from openai import OpenAI

# Initialize OpenAI client using OpenRouter's base URL and API key from environment variable
mxbai = MixedbreadAI(api_key = os.getenv('MIXEDBREAD_API_KEY'))

for review in data['reviews'] :
    response = mxbai.embeddings(
        input = review['comments'],
        model = "mixedbread-ai/mxbai-embed-large-v1",
        normalized = True,
        encoding_format = 'float',
        truncation_strategy = 'end'
    )
    embedding = response.data[0].embedding
    processed_data.append(
    (
        review['title'],  # ID
        embedding,  # Vector
        {  # Metadata
            'review': review['comments'],
            'genre': review['genre'],
            'stars': review['stars'],
            'reviewer': review['reviewer']
        }
    )
)

In [29]:
processed_data[0]

('Inception',
 [0.021820068,
  -0.0030651093,
  -0.006526947,
  0.012512207,
  -0.031219482,
  -0.034240723,
  -0.017333984,
  -0.010650635,
  0.021514893,
  -0.0030059814,
  0.010063171,
  0.0027694702,
  -0.006324768,
  -0.037994385,
  0.019714355,
  -0.02079773,
  -0.048950195,
  -0.018096924,
  -0.07324219,
  0.044891357,
  -0.019012451,
  0.018615723,
  -0.053710938,
  -0.048095703,
  0.024169922,
  0.030899048,
  0.03616333,
  0.019729614,
  0.01889038,
  0.049102783,
  0.029571533,
  0.011268616,
  0.033416748,
  -0.014129639,
  0.0116119385,
  -0.01977539,
  0.03970337,
  -0.04046631,
  0.03414917,
  -0.016281128,
  0.041656494,
  -0.0317688,
  -0.016708374,
  0.0023288727,
  -0.038116455,
  0.005256653,
  0.04168701,
  -0.046936035,
  0.023330688,
  -0.025802612,
  -0.017593384,
  -0.018859863,
  0.04953003,
  0.025024414,
  0.0046806335,
  0.014152527,
  -0.007209778,
  0.016220093,
  -0.0036296844,
  0.04446411,
  0.015571594,
  -0.023620605,
  0.018600464,
  0.014259338,
  

In [30]:
index = pc.Index('rag')
index.upsert(
    vectors= processed_data,
    namespace = 'ns1'
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 29 Aug 2024 23:45:54 GMT', 'Content-Type': 'application/json', 'Content-Length': '104', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '194', 'x-pinecone-request-id': '4087409789179438531', 'x-envoy-upstream-service-time': '52', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 1024 does not match the dimension of the index 1536","details":[]}
